#Import libraries and mount drive

In [2]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingRegressor


import xgboost as xgb
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.impute import SimpleImputer

from sklearn import svm
from sklearn.svm import SVR
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Data pre-processing and feature engineering

In [3]:
players_21 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab files/players_21.csv')


In [4]:
players_22 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab files/players_22.csv')

<ipython-input-4-c0af8ff2dfb1>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_22 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab files/players_22.csv')


In [5]:
players_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 15.9+ MB


In [6]:
columns_list = players_21.columns.tolist()
print(columns_list)

['sofifa_id', 'player_url', 'short_name', 'long_name', 'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'club_team_id', 'club_name', 'league_name', 'league_level', 'club_position', 'club_jersey_number', 'club_loaned_from', 'club_joined', 'club_contract_valid_until', 'nationality_id', 'nationality_name', 'nation_team_id', 'nation_position', 'nation_jersey_number', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'real_face', 'release_clause_eur', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power

In [7]:
#Dropping columns with data sets that have 50% missing values
cols_to_drop = []
for i in players_21.columns:
    missing = np.abs((players_21[i].count() - players_21[i].shape[0])/players_21[i].shape[0] * 100)
    if missing > 50:
        print('{} - {}%'.format(i, round(missing)))
        cols_to_drop.append(i)

players_21.drop(columns=cols_to_drop,inplace=True)
print(players_21.shape)

columns_to_drop = ['sofifa_id', 'player_url', 'player_face_url', 'club_logo_url', 'club_flag_url', 'nation_flag_url']
players_21.drop(columns=columns_to_drop,inplace=True)

club_loaned_from - 96%
nation_team_id - 94%
nation_position - 94%
nation_jersey_number - 94%
player_tags - 93%
player_traits - 56%
goalkeeping_speed - 89%
nation_logo_url - 94%
(18944, 102)


In [8]:
selected_columns = ['overall','potential', 'wage_eur','age','height_cm','weight_kg','preferred_foot', 'international_reputation', 'release_clause_eur','pace',
                    'shooting', 'passing', 'dribbling', 'physic', 'attacking_crossing', 'attacking_finishing','attacking_heading_accuracy',
                    'attacking_short_passing','attacking_volleys','skill_dribbling',
                    'skill_curve', 'skill_fk_accuracy','skill_long_passing', 'skill_ball_control','movement_acceleration',
                    'movement_sprint_speed','movement_agility','movement_reactions','movement_balance', 'power_shot_power','power_stamina',
                    'power_strength','power_long_shots', 'mentality_aggression','mentality_interceptions','mentality_positioning', 'mentality_vision',
                    'mentality_composure','defending_marking_awareness','defending_standing_tackle','defending_sliding_tackle','goalkeeping_diving',
                   'goalkeeping_handling', 'goalkeeping_kicking','goalkeeping_positioning','goalkeeping_reflexes']

players_21_Data= players_21[selected_columns]


In [9]:
# Split dataset into categorical and numeric
categorical_data =players_21_Data.select_dtypes(include=['object'])
numeric_data = players_21_Data.select_dtypes(exclude=['object'])

#imputing numeric
imp=SimpleImputer()
imputed_data=imp.fit_transform(numeric_data)
df_numeric =pd.DataFrame(imputed_data, columns=numeric_data.columns)

#imputing non_numeric
df = pd.DataFrame(categorical_data)
columns=categorical_data.columns
modes = df[columns].mode().iloc[0]
df[columns] = df[columns].fillna(modes)


In [10]:
# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the encoder to the data and transform the data
encoded_data = label_encoder.fit_transform(df)

# The encoded_data variable now contains numerical labels
print(encoded_data)


[0 1 1 ... 1 1 1]


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
df_encoded= pd.DataFrame(encoded_data, columns=categorical_data.columns)
New_players_21 = pd.concat([df_numeric,df_encoded], axis=1)
New_players_21

,overall,potential,wage_eur,age,height_cm,weight_kg,international_reputation,release_clause_eur,pace,shooting,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,preferred_foot
0,93.0,93.0,560000.0,33.0,170.0,72.0,5.0,138400000.0,85.0,92.0,...,96.0,32.0,35.0,24.0,6.0,11.0,15.0,14.0,8.0,0
1,92.0,92.0,220000.0,35.0,187.0,83.0,5.0,75900000.0,89.0,93.0,...,95.0,28.0,32.0,24.0,7.0,11.0,15.0,14.0,11.0,1
2,91.0,91.0,240000.0,31.0,184.0,80.0,4.0,132000000.0,78.0,91.0,...,88.0,35.0,42.0,19.0,15.0,6.0,12.0,8.0,10.0,1
3,91.0,91.0,270000.0,28.0,175.0,68.0,5.0,166500000.0,91.0,85.0,...,93.0,35.0,30.0,29.0,9.0,9.0,15.0,15.0,11.0,1
4,91.0,91.0,370000.0,29.0,181.0,70.0,4.0,161000000.0,76.0,86.0,...,91.0,68.0,65.0,53.0,15.0,13.0,5.0,10.0,13.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,47.0,52.0,1000.0,21.0,177.0,70.0,1.0,57000.0,58.0,23.0,...,40.0,45.0,56.0,47.0,12.0,13.0,8.0,14.0,6.0,1
18940,47.0,53.0,1000.0,21.0,174.0,68.0,1.0,72000.0,58.0,32.0,...,35.0,43.0,42.0,53.0,8.0,8.0,13.0,14.0,10.0,1
18941,47.0,47.0,2000.0,28.0,185.0,79.0,1.0,47000.0,65.0,37.0,...,35.0,38.0,43.0,45.0,8.0,5.0,11.0,5.0,7.0,1
18942,47.0,67.0,500.0,17.0,171.0,58.0,1.0,165000.0,70.0,46.0,...,45.0,18.0,11.0,13.0,11.0,13.0,9.0,9.0,6.0,1


#Checking for maximum correlation


In [12]:
def find_max_corr_features(data, dependent_variable, num_subsets):
    max_corr_subsets = []

    # Correlation matrix
    corr_matrix = data.corr().abs()

    # Column indices of features sorted by correlation with dependent variable
    sorted_features = np.argsort(corr_matrix[dependent_variable])[::-1]

    for _ in range(num_subsets):
        subset = []
        for i in sorted_features:
            if i not in subset:
                subset.append(i)
                if len(subset) == data.shape[1] - 1:
                    break
        max_corr_subsets.append(subset)

    return max_corr_subsets


dependent_variable = 'overall'
num_subsets = 15

max_corr_subsets = find_max_corr_features(New_players_21, dependent_variable, num_subsets)

for i, subset in enumerate(max_corr_subsets):
    print(f"Subset {i + 1}: {subset}")

Subset 1: [0, 26, 36, 10, 1, 7, 11, 2, 28, 35, 16, 12, 21, 3, 9, 22, 6, 19, 13, 31, 32, 20, 29, 18, 17, 30, 34, 15, 14, 37, 33, 25, 38, 39, 24, 23, 8, 5, 27, 45, 4, 43, 44, 41, 40]
Subset 2: [0, 26, 36, 10, 1, 7, 11, 2, 28, 35, 16, 12, 21, 3, 9, 22, 6, 19, 13, 31, 32, 20, 29, 18, 17, 30, 34, 15, 14, 37, 33, 25, 38, 39, 24, 23, 8, 5, 27, 45, 4, 43, 44, 41, 40]
Subset 3: [0, 26, 36, 10, 1, 7, 11, 2, 28, 35, 16, 12, 21, 3, 9, 22, 6, 19, 13, 31, 32, 20, 29, 18, 17, 30, 34, 15, 14, 37, 33, 25, 38, 39, 24, 23, 8, 5, 27, 45, 4, 43, 44, 41, 40]
Subset 4: [0, 26, 36, 10, 1, 7, 11, 2, 28, 35, 16, 12, 21, 3, 9, 22, 6, 19, 13, 31, 32, 20, 29, 18, 17, 30, 34, 15, 14, 37, 33, 25, 38, 39, 24, 23, 8, 5, 27, 45, 4, 43, 44, 41, 40]
Subset 5: [0, 26, 36, 10, 1, 7, 11, 2, 28, 35, 16, 12, 21, 3, 9, 22, 6, 19, 13, 31, 32, 20, 29, 18, 17, 30, 34, 15, 14, 37, 33, 25, 38, 39, 24, 23, 8, 5, 27, 45, 4, 43, 44, 41, 40]
Subset 6: [0, 26, 36, 10, 1, 7, 11, 2, 28, 35, 16, 12, 21, 3, 9, 22, 6, 19, 13, 31, 32, 20, 29,

In [13]:
df = pd.DataFrame(New_players_21)
correlation = df.corr()['overall']
threshold = 0.4  # Adjust the threshold as needed

# Filter out columns with correlation below the threshold
low_corr_features = correlation[abs(correlation) < threshold].index

# Drop the low correlation features from the DataFrame
df_filtered = df.drop(low_corr_features, axis=1)

# Display the resulting DataFrame
df_filtered.columns

Index(['overall', 'potential', 'wage_eur', 'age', 'international_reputation',
       'release_clause_eur', 'shooting', 'passing', 'dribbling', 'physic',
       'attacking_crossing', 'attacking_short_passing', 'skill_curve',
       'skill_long_passing', 'skill_ball_control', 'movement_reactions',
       'power_shot_power', 'power_long_shots', 'mentality_aggression',
       'mentality_vision', 'mentality_composure'],
      dtype='object')

In [15]:
selected_columns = ['overall', 'potential', 'wage_eur', 'age', 'international_reputation',
       'release_clause_eur', 'shooting', 'passing', 'dribbling', 'physic',
       'attacking_crossing', 'attacking_short_passing', 'skill_curve',
       'skill_long_passing', 'skill_ball_control', 'movement_reactions',
       'power_shot_power', 'power_long_shots', 'mentality_aggression',
       'mentality_vision', 'mentality_composure']

X = df_filtered.drop(columns='overall')
y = df_filtered['overall']

#Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
#Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

Training suitable machine learning models with cross-validation

#The Random forest Model

In [17]:
# Initialize the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Perform cross-validation and print the mean accuracy score
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5)
mean_cv_scores_rf=np.mean(cv_scores)
print(f"Random Forest Model CV Mean Accuracy Score: {mean_cv_scores_rf}")

Random Forest Model CV Mean Accuracy Score: 0.9850973023720616


In [18]:
#Make the prediction
y_pred = cross_val_predict(rf_model, X_train, y_train, cv=5)
print(rf_model)
#Measure the model's performance using the Mean absolute error and the Root mean squared error
mae = mean_absolute_error(y_train, y_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

RandomForestRegressor(random_state=42)
MAE: 0.573972286374134
RMSE: 0.8588298415866417


#The Gradient Boosting Model

In [19]:
# Initialize the model
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Perform cross-validation and print the mean accuracy score
cv_scores = cross_val_score(gb_model, X_train, y_train, cv=5)
mean_cv_scores_gr = np.mean(cv_scores)
print(f"Gradient Boosting Model CV Mean Accuracy Score: {mean_cv_scores_gr}")


Gradient Boosting Model CV Mean Accuracy Score: 0.975700110797652


In [21]:
#Make the prediction
y_pred = cross_val_predict(gb_model, X_train, y_train, cv=5)
print(y_pred)
#Measure the model's performance using the Mean absolute error and the Root mean squared error
mae = mean_absolute_error(y_train, y_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

[51.02844576 54.56126107 67.19545862 ... 70.97012699 77.79867232
 59.02925232]
MAE: 0.8177678965718205
RMSE: 1.0965543791591972


#The Support Vector Machine Model

In [ ]:
#Support vector machine model
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score

# Initialize the model
svm_model = SVR(kernel='linear', C=1.0, gamma='scale')

# Perform cross-validation and print the mean accuracy score
cv_scores = cross_val_score(svm_model, X_train, y_train, cv=5)
mean_cv_scores_svm=np.mean(cv_scores)
print(f"Support Vector Machine Model CV Mean Accuracy Score: {mean_cv_scores_svm}")


In [ ]:
#Make the prediction
y_pred = cross_val_predict(svm_model, X_train, y_train, cv=5)
print(y_pred)

#Measure the model's performance using the Mean absolute error and the Root mean squared error
mae = mean_absolute_error(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[89. 87. 90. ... 47. 51. 50.]
MAE: 1.3131862331081081
RMSE: 1.8623305065268465


# The XGBoost model

In [22]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators=100)

xgb_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

average_score = np.mean(xgb_scores)

print("Average 5-Fold Cross-Validation Score: {:.4f}".format(average_score))



Average 5-Fold Cross-Validation Score: -0.7806


In [23]:
xgb_predictions = cross_val_predict(xgb_model, X_train, y_train, cv=5)
print(xgb_predictions)

#Measure the model's performance using the Mean absolute error and the Root mean squared error
mae = mean_absolute_error(y_train, xgb_predictions)
rmse = np.sqrt(mean_squared_error(y_train, xgb_predictions))

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

[50.590855 54.76696  67.04098  ... 69.8503   77.340164 59.001217]
MAE: 0.6370107194528608
RMSE: 0.8835107494665911


#Custom ensemble model: The stacking regressor

Creating the ensemble model and make predictions on the test set

In [24]:
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators=100)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)


# Train the base models
xgb_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)


# Make predictions on the testing set
xgb_preds = xgb_model.predict(X_test)
rf_preds = rf_model.predict(X_test)
gb_preds = gb_model.predict(X_test)

# Combine the predictions (e.g., by averaging them)
ensemble_preds = np.mean([xgb_preds, rf_preds,gb_preds], axis=0)
ensemble_preds

array([60.85379918, 62.9291503 , 57.75548133, ..., 72.64954405,
       63.11990221, 61.43585857])

Calculate our ensemble model's accuracy

In [ ]:
# Calculate the MAE and RMSE

mae = mean_absolute_error(y_test,  ensemble_preds)
rmse = np.sqrt(mean_squared_error(y_test, ensemble_preds))

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

MAE: 0.5826119075994841
RMSE: 0.8179154854478812


#Optimize the ensemble model

Determine the ensemble model's architecture

In [25]:
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
# Create the stacking regressor
stacking_regressor = StackingRegressor(
    estimators=[
        ('xgb', xgb_model),
        ('rf', rf_model),
        ('gb', gb_model)
    ],
    final_estimator=XGBRegressor(n_estimators=100, random_state=42),
    cv=5
)

# Fit the stacking regressor to the training set
stacking_regressor.fit(X_train, y_train)

# Predict the target variable for the test set
stacking_preds = stacking_regressor.predict(X_test)
stacking_preds

array([60.986153, 62.93745 , 57.848763, ..., 73.325714, 63.100933,
       61.51793 ], dtype=float32)

# Perform the random grid search


In [29]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'xgb__n_estimators': [100, 150, 200],
    'xgb__learning_rate': [0.1, 0.05, 0.01],
    'xgb__max_depth': [3, 5, 7],

    'rf__n_estimators': [100, 150, 200],
    'rf__max_features': ['auto', 'sqrt', 'log2'],
    'rf__max_depth': [None, 10, 20],

    'gb__n_estimators': [100, 150, 200],
    'gb__learning_rate': [0.1, 0.05, 0.01],
    'gb__max_depth': [3, 5, 7]
}

# Perform the random grid search
n_iter = 50
random_search = RandomizedSearchCV(
    estimator=stacking_regressor,       # Replace with your model
    param_distributions=param_grid,
    n_iter=n_iter,
    scoring='neg_mean_squared_error',  # Choose an appropriate scoring metric
    cv=5,  # Number of cross-validation folds
    n_jobs=-1,  # Use all available CPU cores for parallel processing
    random_state=42  # Set a random seed for reproducibility
)

In [30]:
random_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors

RandomizedSearchCV(cv=5,
                   estimator=StackingRegressor(cv=5,
                                               estimators=[('xgb',
                                                            XGBRegressor(base_score=None,
                                                                         booster=None,
                                                                         callbacks=None,
                                                                         colsample_bylevel=None,
                                                                         colsample_bynode=None,
                                                                         colsample_bytree=None,
                                                                         device=None,
                                                                         early_stopping_rounds=None,
                                                                         enable_categorical=False,
                                                                         eval_metric=None,
                                                                         feature_types=None,
                                                                         gamma=None,
                                                                         grow_policy=None,
                                                                         importance_type=None,
                                                                         i...
                   param_distributions={'gb__learning_rate': [0.1, 0.05, 0.01],
                                        'gb__max_depth': [3, 5, 7],
                                        'gb__n_estimators': [100, 150, 200],
                                        'rf__max_depth': [None, 10, 20],
                                        'rf__max_features': ['auto', 'sqrt',
                                                             'log2'],
                                        'rf__n_estimators': [100, 150, 200],
                                        'xgb__learning_rate': [0.1, 0.05, 0.01],
                                        'xgb__max_depth': [3, 5, 7],
                                        'xgb__n_estimators': [100, 150, 200]},
                   random_state=42, scoring='neg_mean_squared_error')

In [31]:
best_model = random_search.best_estimator_
best_params =random_search.best_params_

y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.6237859791984853


#Test ensemble model on the Players_22 data

Load the trained ensemble model and test it's performance on the Players_22 data

In [32]:
# Test the model on players_22
data= ['overall', 'potential', 'wage_eur', 'age', 'international_reputation',
       'release_clause_eur', 'shooting', 'passing', 'dribbling', 'physic',
       'attacking_crossing', 'attacking_short_passing', 'skill_curve',
       'skill_long_passing', 'skill_ball_control', 'movement_reactions',
       'power_shot_power', 'power_long_shots', 'mentality_aggression',
       'mentality_vision', 'mentality_composure']

players_22_Data= players_22[data]

In [33]:
# Split dataset into categorical and numeric
numeric_data = players_22_Data.select_dtypes(exclude=['object'])

#imputing numeric
imp=SimpleImputer()
imputed_data=imp.fit_transform(numeric_data)
df_numeric =pd.DataFrame(imputed_data, columns=numeric_data.columns)




In [34]:
X =df_numeric.drop(columns='overall')
y = df_numeric['overall']

#Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

test_pred = best_model.predict(X_test)
test_pred

array([64.51959 , 61.497246, 56.422565, ..., 63.354374, 67.23068 ,
       67.727974], dtype=float32)

In [35]:
mse = mean_squared_error(y_test ,test_pred )
print("Mean Squared Error:", mse)


Mean Squared Error: 3.655760300009603


In [37]:
import pickle
# Use pickle to save the trained model
with open('best_model.plk', 'wb') as f:
    pickle.dump(best_model, f)


'''# Use pickle to load the saved model
with open('iris_model.pkl', 'rb') as f:
    clf_loaded = pickle.load(f)'''

"# Use pickle to load the saved model\nwith open('iris_model.pkl', 'rb') as f:\n    clf_loaded = pickle.load(f)"